In [1]:
import trimesh
import numpy as np
import sys
import h5py
import point_cloud_utils as pcu
from igl import decimate
from glob import glob
from tqdm.notebook import tqdm
from pathlib import Path

# Path to workspace
sys.path.insert(0, '/workspace/dense-self-supervised-representation-learning-for-3D-shapes/')
from workspace.datasets.transforms import PointCloudNormalize

import k3d

In [2]:
aliens = '/workspace/coseg/aliens/'
chairs = '/workspace/coseg/chairs/'
vases = '/workspace/coseg/vases/'

In [3]:
cat2label = {
    'aliens': 0,
    'vases': 1,
    'chairs': 2
}


def prepare_mesh(mesh_path, faces_labels, n_points=1024):
    old_v, old_f = pcu.load_mesh_vf(mesh_path)
    old_v = PointCloudNormalize()(old_v)
    
    res, v, f, f_to_f, _ = decimate(old_v, old_f, 500)
    faces_labels = faces_labels[f_to_f]

    fi, bc = pcu.sample_mesh_poisson_disk(v, f, int(n_points * 1.2))

    v_poisson = pcu.interpolate_barycentric_coords(f, fi, bc, v)

    if v_poisson.shape[0] > n_points:
        perm = np.random.choice(v_poisson.shape[0], n_points, replace=False)
        points = v_poisson[perm]
        face_index = fi[perm]
    else:
        print(f'Error: {mesh_path}')
        raise ValueError()

    return v, f, faces_labels, face_index, points

In [28]:
def write_to_h5(root_dir, output_path, n_points=1024):
    root_path = Path(root_dir)
    files_path = root_path / 'shapes' / '*'
    files_paths = glob(str(files_path))
    
    data_len = len(files_paths)

    with h5py.File(output_path, 'w') as out:
        out.create_dataset('faces', dtype=h5py.vlen_dtype('i4'), shape=(data_len,))
        out.create_dataset('faces_labels', dtype=h5py.vlen_dtype('i4'), shape=(data_len,))
        out.create_dataset('vertices', dtype=h5py.vlen_dtype('f4'), shape=(data_len,))
        out.create_dataset('face_index', dtype='i4', shape=(data_len, n_points))
        out.create_dataset('points', dtype='f4', shape=(data_len, n_points, 3))
        out.create_dataset('points_labels', dtype='i4', shape=(data_len, n_points))
        out.create_dataset('instance_labels', dtype='i4', shape=(data_len,))
    
        for i, file_path in tqdm(enumerate(files_paths), total=len(files_paths)):
            parts = file_path.split('/')
            category = parts[-3]
            idx = parts[-1].replace('.off', '')
            label_path = root_path / 'gt' / f'{idx}.seg'
            faces_labels = np.loadtxt(label_path, dtype=np.int32)
            faces_labels -= 1

            v, f, faces_labels, fi, pts = prepare_mesh(file_path, faces_labels)
            out['vertices'][i] = v.flatten()
            out['faces'][i] = f.flatten()
            out['faces_labels'][i] = faces_labels
            out['face_index'][i] = fi
            out['points'][i] = pts
            out['points_labels'][i] = faces_labels[fi]
            out['instance_labels'][i] = cat2label[category]
    

In [35]:
write_to_h5(aliens, 'aliens.h5')

  0%|          | 0/196 [00:00<?, ?it/s]

In [36]:
write_to_h5(chairs, 'chairs.h5')

  0%|          | 0/393 [00:00<?, ?it/s]

In [37]:
write_to_h5(vases, 'vases.h5')

  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
path = '/workspace/coseg/aliens/shapes/1.off'
mesh = trimesh.load_mesh('/workspace/coseg/aliens/shapes/1.off')
gt = np.loadtxt('/workspace/coseg/aliens/gt/1.seg', dtype=np.int32)

In [ ]:
files_paths = glob(f'/workspace/coseg/*/shapes/*')
root_dir = '/workspace/coseg'
    
for i, file_path in tqdm(enumerate(files_paths), total=900):
    parts = file_path.split('/')
    category = parts[-3]
    idx = parts[-1].replace('.off', '')
    label_path = f'{root_dir}/{category}/gt/{idx}.seg'
    faces_labels = np.loadtxt(label_path, dtype=np.int32)
    
    mesh = trimesh.load_mesh(file_path)
    
    res, _, _, _, _ = decimate(mesh.vertices, mesh.faces, 500)

    if res != True:
        print(file_path)

In [ ]:
!rm /workspace/coseg/chairs/shapes/205.off
!rm /workspace/coseg/chairs/shapes/228.off
!rm /workspace/coseg/chairs/shapes/245.off
!rm /workspace/coseg/chairs/shapes/225.off
!rm /workspace/coseg/chairs/shapes/208.off
!rm /workspace/coseg/chairs/shapes/252.off
!rm /workspace/coseg/chairs/shapes/248.off
!rm /workspace/coseg/aliens/shapes/100.off
!rm /workspace/coseg/aliens/shapes/80.off
!rm /workspace/coseg/aliens/shapes/117.off
!rm /workspace/coseg/aliens/shapes/57.off

In [ ]:
_, v, f, fi, vi = decimate(np.array(mesh.vertices), np.array(mesh.faces), 500)

In [ ]:
new_gt = gt[fi]

In [ ]:
k3d.mesh(v, f, triangles_attribute=new_gt)

In [ ]:
k3d.mesh(mesh.vertices, mesh.faces, triangles_attribute=gt)

In [ ]:
write_to_h5('../../datasets/coseg', '../../datasets/hdfs/coseg.h5')